### 1. Analyseurs et Filtres

Les analyseurs et filtres sont essentiels dans Elasticsearch pour transformer du texte brut en un format structuré pour l'indexation et la recherche.

#### Analyseurs (analyzers)

Un analyseur est une combinaison de trois composants :

- Tokenizer : Découpe le texte en termes.
- Filters : Modifient les termes extraits.
- Character Filters : Modifient le texte avant le tokenization.


Exemples :

- ```standard```: Tokenizer par défaut qui divise les mots selon la casse et la ponctuation.
- ```edge_ngram```: Utilisé pour l’autocomplétion, il génère des sous-chaînes d’un mot.
- ```synonym```: Remplace les termes par des synonymes définis.
- ```shingle```: Génère des combinaisons de mots consécutifs pour améliorer la recherche de phrases.

### 2. Scoring et Ranking des Documents
Le score de pertinence détermine l’ordre des résultats de recherche.

#### BM25 (Okapi BM25)
- Algorithme par défaut de scoring dans Elasticsearch.
- Fonctionne sur le principe de la fréquence des termes et de la longueur du document.
- Avantage : pondère mieux les documents longs et les mots-clés fréquents.

#### ```function_score```
Permet de modifier le score de base avec des fonctions personnalisées (ex : popularité, date récente, etc.).
- Exemple :

```json
{
  "query": {
    "function_score": {
      "query": { "match": { "title": "Elasticsearch" } },
      "functions": [
        { "field_value_factor": { "field": "popularity", "factor": 1.2, "modifier": "log1p" } }
      ]
    }
  }
}
```

#### Boosting
Permet d’augmenter ou de diminuer la pertinence d’un document.
- Exemple :

```json
{
  "query": {
    "boosting": {
      "positive": { "match": { "title": "Elasticsearch" } },
      "negative": { "match": { "title": "obsolete" } },
      "negative_boost": 0.5
    }
  }
}
```

### 3. Requêtes Booléennes Avancées
Les requêtes booléennes permettent de combiner plusieurs conditions de recherche.

#### Match Phrase (```match_phrase```)
Recherche une expression exacte en maintenant l'ordre des mots.
- Exemple :

```json
{ "query": { "match_phrase": { "description": "moteur de recherche" } } }
```

#### Fuzzy Search (```fuzzy```)
Recherche en tolérant des fautes de frappe.
- Exemple :

```json
{ "query": { "match": { "name": { "query": "Elastisearch", "fuzziness": "AUTO" } } } }
```

### Multi Match (```multi_match```)
Recherche sur plusieurs champs simultanément.
- Exemple :

```json
{ "query": { "multi_match": { "query": "search engine", "fields": ["title", "description"] } } }
```

### 4. Suggestions et Autocomplétion
Elasticsearch propose plusieurs méthodes pour la suggestion et l'autocomplétion.

#### Completion Suggester
Utilisé pour l'autocomplétion rapide.
- Exemple :

```json
{ "query": { "match": { "suggest": "Elast" } } }
```

#### Phrase Suggester
Propose des corrections orthographiques basées sur le contexte.
- Exemple :

```json
{
  "suggest": {
    "text": "Elastisearch",
    "my_suggestion": {
      "phrase": {
        "field": "title",
        "size": 1
      }
    }
  }
}
```

### 5. Sharding et Réplication pour la Scalabilité
Elasticsearch utilise sharding et réplication pour gérer de gros volumes de données.

- Sharding : Découpe un index en plusieurs morceaux.
Exemple : Un index de 1 milliard de documents peut être découpé en 5 shards.
- Réplication : Copie des shards pour la redondance et la performance.
Exemple : Un shard peut avoir 2 copies répliquées sur d'autres nœuds.
#### Optimisation des Shards
- Trop de shards = surcharge mémoire.
- Trop peu de shards = sous-exploitation des ressources.


### 6. Différence entre Match, Term et Bool Queries
- Match Query : Recherche en utilisant un analyseur (utilisé pour le texte).

```json
{ "query": { "match": { "title": "elasticsearch" } } }
```

- Term Query : Recherche un mot exact sans analyse.

```json
{ "query": { "term": { "title": "Elasticsearch" } } }
```

- Bool Query : Combine plusieurs conditions (must, should, must_not).

```json
{
  "query": {
    "bool": {
      "must": [ { "match": { "title": "Elasticsearch" } } ],
      "must_not": [ { "match": { "category": "old" } } ],
      "should": [ { "match": { "popularity": "high" } } ]
    }
  }
}
```

### 7. Création d’un Pipeline d’Ingestion pour Enrichir les Documents
Les ingest pipelines permettent de transformer les documents avant indexation.

#### Exemple de pipeline d’enrichissement
Ajout d’un champ avec la longueur du texte :

```json
PUT _ingest/pipeline/text_length
{
  "description": "Ajoute la longueur du texte",
  "processors": [
    {
      "script": {
        "source": "ctx.text_length = ctx.content.length()"
      }
    }
  ]
}
```

Appliquer ce pipeline :

```json
PUT my_index/_doc/1?pipeline=text_length
{
  "content": "Elasticsearch est un moteur de recherche puissant."
}
```

### 8. Optimisation des Performances
#### Caching
- Elasticsearch met en cache les requêtes fréquentes pour améliorer la rapidité.
- Activer le cache pour certaines queries :

```json
{
  "query": {
    "match": { "title": "Elasticsearch" }
  },
  "_cache": true
}
```

#### Tuning des Index
- Nombre de shards optimal : Un bon ratio est 1 shard par 50 Go de données.
- Compression des index : Utiliser best_compression pour réduire l’espace disque.
#### Optimisation des Requêtes
- Privilégier filter (pas de scoring) sur query si le scoring n’est pas nécessaire.
- Réduire la taille de from et size dans search pour éviter de récupérer trop de documents en une seule requête.